# This notebook compares sequence similarity between groups of TEs
- follow the same procedure with 4a
- computed the similarity scores for the top certain numbers up regulated elements in turner WT organoids data
- compute the score for randomly selected elements that might not be differential

In [1]:
from pyfaidx import Fasta
import pandas as pd
from Bio.Seq import Seq
from pathlib import Path

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from Bio import SeqIO, pairwise2
from itertools import combinations
from multiprocessing import Pool, cpu_count
from pathlib import Path

/home/lucy/anaconda3/envs/qc_pipe/lib/python3.8/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [38]:
import os

In [3]:
#from pairwise2 import format_alignment

In [4]:
import time

In [5]:
import sequence_comparison as seqComp

## load data

In [7]:
wt_up = pd.read_csv("./compare_dTE_overlap/squire_sig_down.csv",header = 0, index_col = 0 )

In [19]:
wt_up['df_index_repName'] = wt_up.index.astype(str) + "_" + wt_up['repName']

In [15]:
wt_up.shape

(33, 18)

In [9]:
squire_indiv = pd.read_csv("/home/lucy/Dropbox/turner_organoid_bulk_RNA_local/Deseq_indiv_dTE_results.csv", header = 0, index_col = 0 )

In [10]:
squire_indiv.head()

,TE_ID,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,element_type,TE_name,TE_index,repName,repFamily,repClass,genoName,genoStart,genoEnd,strand
0,"chr13|73480020|73480091|PB1D9:Alu:SINE|268|-,.",9.000885,-0.489762,0.629289,-0.778278,0.800222,1.0,TE,PB1D9:Alu:SINE,PB1D9:Alu:SINE_268,PB1D9,Alu,SINE,chr13,73480020,73480091,-
1,"chr5|117467874|117468005|RSINE1:B4:SINE|183|-,.",0.730917,-2.916743,3.081604,-0.946502,0.368835,1.0,TE,RSINE1:B4:SINE,RSINE1:B4:SINE_183,RSINE1,B4,SINE,chr5,117467874,117468005,-
2,"chr3|96619212|96619284|ID4:ID:SINE|194|+,.",5.772588,0.121464,0.852163,0.142536,0.942801,1.0,TE,ID4:ID:SINE,ID4:ID:SINE_194,ID4,ID,SINE,chr3,96619212,96619284,+
3,"chr15|93388865|93388965|PB1:Alu:SINE|227|-,.",32.028966,-0.216654,0.390147,-0.555314,0.978578,1.0,TE,PB1:Alu:SINE,PB1:Alu:SINE_227,PB1,Alu,SINE,chr15,93388865,93388965,-
4,"chr11|94175967|94176153|B2_Mm2:B2:SINE|98|-,.",7.127690,0.305487,0.591882,0.516129,0.893386,1.0,TE,B2_Mm2:B2:SINE,B2_Mm2:B2:SINE_98,B2_Mm2,B2,SINE,chr11,94175967,94176153,-


In [11]:
squire_df_ind = squire_indiv.index.astype(str) + "_" + squire_indiv['repName']

In [12]:
squire_indiv['df_index_repName'] = squire_df_ind

In [13]:
squire_indiv.head(2)

,TE_ID,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,element_type,TE_name,TE_index,repName,repFamily,repClass,genoName,genoStart,genoEnd,strand,df_index_repName
0,"chr13|73480020|73480091|PB1D9:Alu:SINE|268|-,.",9.000885,-0.489762,0.629289,-0.778278,0.800222,1.0,TE,PB1D9:Alu:SINE,PB1D9:Alu:SINE_268,PB1D9,Alu,SINE,chr13,73480020,73480091,-,0_PB1D9
1,"chr5|117467874|117468005|RSINE1:B4:SINE|183|-,.",0.730917,-2.916743,3.081604,-0.946502,0.368835,1.0,TE,RSINE1:B4:SINE,RSINE1:B4:SINE_183,RSINE1,B4,SINE,chr5,117467874,117468005,-,1_RSINE1


In [14]:
mm39_ref_path = '/home/lucy/Dropbox/mouse_ref_genome_mm39/mm39.fa'

## Get similarity score for WT up sequences
- top 10, 20, and all (33)

### get sequences

In [20]:
wt_up.sort_values( by = 'log2FoldChange', ascending = True , inplace= True )
wt_up.head()

,TE_ID,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,element_type,TE_name,TE_index,repName,repFamily,repClass,genoName,genoStart,genoEnd,strand,df_index_repName
4485,"chrY|1046951|1048785|Lx2B2:L1:LINE|166|+,.",41.184881,-8.735873,1.051304,-8.309562,9.309039e-14,1.761051e-11,TE,Lx2B2:L1:LINE,Lx2B2:L1:LINE_166,Lx2B2,L1,LINE,chrY,1046951,1048785,+,4485_Lx2B2
6115,chr19|36949092|36949293|ORR1B1-int:ERVL-MaLR:L...,37.460687,-8.598821,1.054039,-8.157969,2.813290e-13,5.170024e-11,TE,ORR1B1-int:ERVL-MaLR:LTR,ORR1B1-int:ERVL-MaLR:LTR_203,ORR1B1-int,ERVL-MaLR,LTR,chr19,36949092,36949293,+,6115_ORR1B1-int
249,chr13|68590731|68597820|RLTR4_MM-int:LTR:LTR|3...,33.623029,-8.445261,1.049767,-8.044888,6.595240e-13,1.134240e-10,TE,RLTR4_MM-int:LTR:LTR,RLTR4_MM-int:LTR:LTR_321,RLTR4_MM-int,LTR,LTR,chr13,68590731,68597820,-,249_RLTR4_MM-int
5703,"chr19|36948880|36949059|B3:B2:SINE|210|+,.",33.652405,-8.443296,1.066800,-7.914598,1.505438e-12,2.495612e-10,TE,B3:B2:SINE,B3:B2:SINE_210,B3,B2,SINE,chr19,36948880,36949059,+,5703_B3
10291,"chr19|37212352|37212822|Lx8:L1:LINE|254|+,.",33.476036,-8.439185,1.050746,-8.031615,7.212397e-13,1.233780e-10,TE,Lx8:L1:LINE,Lx8:L1:LINE_254,Lx8,L1,LINE,chr19,37212352,37212822,+,10291_Lx8


In [21]:
wt_top10_df = wt_up.iloc[0:10][[ 'genoName', 'genoStart', 'genoEnd', 'strand', 'df_index_repName'] ] 
wt_top20_df = wt_up.iloc[0:20][[ 'genoName', 'genoStart', 'genoEnd', 'strand', 'df_index_repName'] ]
wt_top_all = wt_up[[ 'genoName', 'genoStart', 'genoEnd', 'strand', 'df_index_repName'] ]

In [26]:
wt_top10_df.head()

,genoName,genoStart,genoEnd,strand,df_index_repName
4485,chrY,1046951,1048785,+,4485_Lx2B2
6115,chr19,36949092,36949293,+,6115_ORR1B1-int
249,chr13,68590731,68597820,-,249_RLTR4_MM-int
5703,chr19,36948880,36949059,+,5703_B3
10291,chr19,37212352,37212822,+,10291_Lx8


In [23]:
# top 10 df
mm39_ref_path = '/home/lucy/Dropbox/mouse_ref_genome_mm39/mm39.fa'
seqComp.extract_ucsc_bedlike_regions( 
    genome_fasta= mm39_ref_path,     
    regions_df= wt_top10_df,      
    output_fasta= "sequence_similarity_input/" + "wt_top10_df.fasta",
)

✅ Extracted sequences saved to: sequence_similarity_input/wt_top10_df.fasta


In [24]:
seqComp.extract_ucsc_bedlike_regions( 
    genome_fasta= mm39_ref_path,     
    regions_df= wt_top20_df,      
    output_fasta= "sequence_similarity_input/" + "wt_top20_df.fasta",
)

✅ Extracted sequences saved to: sequence_similarity_input/wt_top20_df.fasta


In [25]:
seqComp.extract_ucsc_bedlike_regions( 
    genome_fasta= mm39_ref_path,     
    regions_df= wt_top_all,      
    output_fasta= "sequence_similarity_input/" + "wt_top_all_df.fasta",
)

✅ Extracted sequences saved to: sequence_similarity_input/wt_top_all_df.fasta


### compute similarity scores

In [27]:
save_path = "./sequence_similarity_results/"
input_path = "./sequence_similarity_input/"

In [28]:
fnames = [ 'wt_top10_df.fasta' , 'wt_top20_df.fasta', 'wt_top_all_df.fasta' ]

In [29]:
for f in fnames:
    print(f"Starting file {f}")
    
    TIME_S = time.time()
    
    seqComp.process_group(input_path + f, out_dir= save_path, n_jobs=4,plot_heatmap=True )
    TIME_E = time.time()
    
    TOTAL_TIME = (TIME_E - TIME_S)/60
    print( f"File {f} time used: {TOTAL_TIME:.1f} min" )

Starting file wt_top10_df.fasta
Loaded 10 sequences from ./sequence_similarity_input/wt_top10_df.fasta
Average identity: 24.43% ± 18.15%
File wt_top10_df.fasta time used: 0.1 min
Starting file wt_top20_df.fasta
Loaded 20 sequences from ./sequence_similarity_input/wt_top20_df.fasta
Average identity: 27.33% ± 19.27%
File wt_top20_df.fasta time used: 0.2 min
Starting file wt_top_all_df.fasta
Loaded 33 sequences from ./sequence_similarity_input/wt_top_all_df.fasta
Average identity: 29.44% ± 19.74%
File wt_top_all_df.fasta time used: 0.4 min


## randomly selected elements
- 10, 20, 50, two selections each

In [35]:
num_selection = [10, 20, 33, 50 ]
rand_state = [608, 504, 2, 10 ]

In [33]:
squire_indiv.head()

,TE_ID,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,element_type,TE_name,TE_index,repName,repFamily,repClass,genoName,genoStart,genoEnd,strand,df_index_repName
0,"chr13|73480020|73480091|PB1D9:Alu:SINE|268|-,.",9.000885,-0.489762,0.629289,-0.778278,0.800222,1.0,TE,PB1D9:Alu:SINE,PB1D9:Alu:SINE_268,PB1D9,Alu,SINE,chr13,73480020,73480091,-,0_PB1D9
1,"chr5|117467874|117468005|RSINE1:B4:SINE|183|-,.",0.730917,-2.916743,3.081604,-0.946502,0.368835,1.0,TE,RSINE1:B4:SINE,RSINE1:B4:SINE_183,RSINE1,B4,SINE,chr5,117467874,117468005,-,1_RSINE1
2,"chr3|96619212|96619284|ID4:ID:SINE|194|+,.",5.772588,0.121464,0.852163,0.142536,0.942801,1.0,TE,ID4:ID:SINE,ID4:ID:SINE_194,ID4,ID,SINE,chr3,96619212,96619284,+,2_ID4
3,"chr15|93388865|93388965|PB1:Alu:SINE|227|-,.",32.028966,-0.216654,0.390147,-0.555314,0.978578,1.0,TE,PB1:Alu:SINE,PB1:Alu:SINE_227,PB1,Alu,SINE,chr15,93388865,93388965,-,3_PB1
4,"chr11|94175967|94176153|B2_Mm2:B2:SINE|98|-,.",7.127690,0.305487,0.591882,0.516129,0.893386,1.0,TE,B2_Mm2:B2:SINE,B2_Mm2:B2:SINE_98,B2_Mm2,B2,SINE,chr11,94175967,94176153,-,4_B2_Mm2


In [37]:
# save fasta
for i, num_s in enumerate( num_selection ):

    squire_subset = squire_indiv[[ 'genoName', 'genoStart', 'genoEnd', 'strand', 'df_index_repName'] ] 
    rand_df1 = squire_subset.sample( n = num_s, random_state= rand_state[i] )
    rand_df2 = squire_subset.sample( n = num_s, random_state= rand_state[i]*2 )
    rand_df3 = squire_subset.sample( n = num_s, random_state= rand_state[i]*3 )

    save_name = [ f"rand{num_s}_{rep}.fasta" for rep in [1,2,3 ] ]
    rand_df_ls = [rand_df1, rand_df2, rand_df3 ] 

    for j in range(len(save_name)):
        #print( "sequence_similarity_input/" + save_name[j] )
        seqComp.extract_ucsc_bedlike_regions( 
            genome_fasta= mm39_ref_path,     
            regions_df= rand_df_ls[j],      
            output_fasta= "sequence_similarity_input/" + save_name[j]
        )
        

    

✅ Extracted sequences saved to: sequence_similarity_input/rand10_1.fasta
✅ Extracted sequences saved to: sequence_similarity_input/rand10_2.fasta
✅ Extracted sequences saved to: sequence_similarity_input/rand10_3.fasta
✅ Extracted sequences saved to: sequence_similarity_input/rand20_1.fasta
✅ Extracted sequences saved to: sequence_similarity_input/rand20_2.fasta
✅ Extracted sequences saved to: sequence_similarity_input/rand20_3.fasta
✅ Extracted sequences saved to: sequence_similarity_input/rand33_1.fasta
✅ Extracted sequences saved to: sequence_similarity_input/rand33_2.fasta
✅ Extracted sequences saved to: sequence_similarity_input/rand33_3.fasta
✅ Extracted sequences saved to: sequence_similarity_input/rand50_1.fasta
✅ Extracted sequences saved to: sequence_similarity_input/rand50_2.fasta
✅ Extracted sequences saved to: sequence_similarity_input/rand50_3.fasta


In [40]:
rand_fnames = pd.Series( os.listdir("./sequence_similarity_input") )
rand_fnames= rand_fnames[rand_fnames.str.startswith('rand') ]
rand_fnames

1     rand10_2.fasta
3     rand20_1.fasta
5     rand20_2.fasta
6     rand33_1.fasta
7     rand33_2.fasta
8     rand50_2.fasta
10    rand50_3.fasta
11    rand50_1.fasta
15    rand10_1.fasta
17    rand33_3.fasta
18    rand10_3.fasta
19    rand20_3.fasta
dtype: object

In [41]:
for f in rand_fnames:
    print(f"Starting file {f}")
    
    TIME_S = time.time()
    
    seqComp.process_group(input_path + f, out_dir= save_path, n_jobs=4,plot_heatmap=True )
    TIME_E = time.time()
    
    TOTAL_TIME = (TIME_E - TIME_S)/60
    print( f"File {f} time used: {TOTAL_TIME:.1f} min\n" )

Starting file rand10_2.fasta
Loaded 10 sequences from ./sequence_similarity_input/rand10_2.fasta
Average identity: 33.86% ± 19.78%
File rand10_2.fasta time used: 0.0 min

Starting file rand20_1.fasta
Loaded 20 sequences from ./sequence_similarity_input/rand20_1.fasta
Average identity: 39.61% ± 17.91%
File rand20_1.fasta time used: 0.0 min

Starting file rand20_2.fasta
Loaded 20 sequences from ./sequence_similarity_input/rand20_2.fasta
Average identity: 34.87% ± 20.68%
File rand20_2.fasta time used: 0.1 min

Starting file rand33_1.fasta
Loaded 33 sequences from ./sequence_similarity_input/rand33_1.fasta
Average identity: 39.72% ± 15.24%
File rand33_1.fasta time used: 0.0 min

Starting file rand33_2.fasta
Loaded 33 sequences from ./sequence_similarity_input/rand33_2.fasta
Average identity: 36.28% ± 17.40%
File rand33_2.fasta time used: 0.1 min

Starting file rand50_2.fasta
Loaded 50 sequences from ./sequence_similarity_input/rand50_2.fasta
Average identity: 38.69% ± 17.97%
File rand50_2.